# Product Confirmation Workflow

This notebook downloads DIST-ALERT products from S3, unzips them, and runs the confirmation workflow.

In [1]:
import pandas as pd
import shutil
import zipfile
import requests
from pathlib import Path
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
from dist_s1 import run_sequential_confirmation_of_dist_products_workflow
from utils import unzip_dist_s1_prod, wrap_run_sequential_confirmation_of_dist_products_workflow
import multiprocessing


/mnt/trappist-r0/cmarshak/mambaforge/envs/dist-s1-val/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tmp_dir =  Path('tmp')
unconfirmed_products_dir =  Path('unconfirmed_products')
confirmed_products_dir =  Path('confirmed_products')

tmp_dir.mkdir(exist_ok=True)
unconfirmed_products_dir.mkdir(exist_ok=True)
confirmed_products_dir.mkdir(exist_ok=True)

In [3]:
# Load the test products CSV
csv_path = Path('val_products_minus_one.csv')
df = pd.read_csv(csv_path)
df.head()

,zip_url,browse_url,product_request_time,processing_duration,high_confidence_alert_threshold,mgrs_tile_id,post_date_buffer_days,stride_for_norm_param_estimation,n_workers_for_norm_param_estimation,delta_lookback_days_mw,...,model_source,memory_strategy,batch_size_for_norm_param_estimation,post_date,track_number,low_confidence_alert_threshold,n_workers_for_despeckling,device,max_pre_imgs_per_burst_mw,model_compilation
0,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:50+00:00,1492.944,4.5,45TUK,1,7,4,none,...,transformer_optimized,high,32,2024-05-05,114,2.5,4,best,none,False
1,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:50+00:00,778.462,4.5,45TUK,1,7,4,none,...,transformer_optimized,high,32,2024-10-25,12,2.5,4,best,none,False
2,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:50+00:00,1609.243,4.5,45TUK,1,7,4,none,...,transformer_optimized,high,32,2024-11-13,114,2.5,4,best,none,False
3,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:50+00:00,590.068,4.5,45TUK,1,7,4,none,...,transformer_optimized,high,32,2024-12-12,12,2.5,4,best,none,False
4,https://hyp3-tibet-jpl-test-contentbucket-hrat...,https://hyp3-tibet-jpl-test-contentbucket-hrat...,2025-09-30T18:19:50+00:00,791.816,4.5,45TUK,1,7,4,none,...,transformer_optimized,high,32,2024-08-14,12,2.5,4,best,none,False


In [4]:
def download_file(url, destination_path):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open(destination_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)
    
    return destination_path


In [5]:
download_tasks = []
for _, row in df.iterrows():
    url = row['zip_url']
    filename = Path(url).name
    zip_path = tmp_dir / filename
    download_tasks.append((url, zip_path))

# Download all files concurrently
downloaded_files = []
with ThreadPoolExecutor(max_workers=10) as executor:
    with tqdm(total=len(download_tasks), desc="Downloading files", unit="file") as pbar:
        future_to_task = {executor.submit(download_file, url, dest_path): dest_path for url, dest_path in download_tasks}
        
        for future in future_to_task:
            file_path = future.result()
            downloaded_files.append(file_path)
            pbar.update(1)

print(f"Downloaded {len(downloaded_files)} files")

Downloaded 5303 files


# Unzip

In [2]:
downloaded_zips = list(Path('tmp/').glob('*.zip'))
len(downloaded_zips)

5303

In [10]:
num_processes = 5 
print('Total processes: ', multiprocessing.cpu_count())
print(f"Using {num_processes} processes for unzipping.")

with multiprocessing.Pool(processes=num_processes) as pool:
    results = pool.imap(unzip_dist_s1_prod, downloaded_zips[:])
    for _ in tqdm(results, total=len(downloaded_zips), desc="Unzipping Files"):
        pass

Total processes:  224
Using 25 processes for unzipping.


Unzipping Files: 100%|█| 5303/5303 [20:41<00:00,


In [4]:
subdirs = list(Path('unconfirmed_products').glob('*/'))
mgrs_tiles_unzipped = [subdir.name for subdir in subdirs]
mgrs_tiles_unzipped[:3]

['45TUK', '21JTF', '33QTU']

In [5]:
# cleanup_temp = True
# if cleanup_temp:
#     shutil.rmtree(tmp_dir)

# Confirmation

In [6]:
# %%time

# for mgrs_tile_id in tqdm(mgrs_tiles_unzipped[2:]):
#     # Run the confirmation workflow
#     run_sequential_confirmation_of_dist_products_workflow(
#         unconfirmed_products_dir / mgrs_tile_id, 
#         confirmed_products_dir / mgrs_tile_id
#     )

In [7]:
with multiprocessing.Pool(processes=10) as pool:
    results = pool.imap(wrap_run_sequential_confirmation_of_dist_products_workflow, mgrs_tiles_unzipped[:])
    for _ in tqdm(results, total=len(mgrs_tiles_unzipped), desc="Confirming Products"):
        pass

Confirming 58 products: 100%|██| 58/58 [18:49<00:00, 19.47s/it]
Confirming 121 products: 100%|█| 121/121 [46:11<00:00, 22.90s/i
Confirming 25 products: 100%|██| 25/25 [06:05<00:00, 14.63s/it]
Confirming 121 products: 100%|█| 121/121 [32:28<00:00, 16.10s/i
Confirming 39 products: 100%|██| 39/39 [09:36<00:00, 14.77s/it]
Confirming 119 products: 100%|█| 119/119 [24:56<00:00, 12.57s/i
Confirming 54 products: 100%|██| 54/54 [12:45<00:00, 14.18s/it]
Confirming 121 products: 100%|█| 121/121 [31:17<00:00, 15.52s/i
Confirming 65 products: 100%|██| 65/65 [16:17<00:00, 15.03s/it]
Confirming 180 products: 100%|█| 180/180 [43:03<00:00, 14.35s/i
Confirming 30 products: 100%|██| 30/30 [05:04<00:00, 10.14s/it]]
Confirming 150 products: 100%|█| 150/150 [40:46<00:00, 16.31s/i
Confirming 60 products: 100%|██| 60/60 [14:26<00:00, 14.44s/it]
Confirming 210 products: 100%|█| 210/210 [57:35<00:00, 16.46s/i
Confirming 99 products: 100%|██| 99/99 [23:42<00:00, 14.37s/it]
Confirming 122 products: 100%|█| 122/12